<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/all_seeds_batch_500_50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
mnist_trainset = datasets.MNIST(root='./data', train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = datasets.MNIST(root='./data', 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

train_dataloader = torch.utils.data.DataLoader(mnist_trainset, 
                                               batch_size=500, 
                                               shuffle=True)

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [4]:
# ************* modify this section for later use *************
# Define the model 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # modify this section for later use 
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 10)
        self.sigmoid12  = torch.nn.Sigmoid()

        self.layer_activations = dict()

    def forward(self, x):

        # modify this section for later use 
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid12(x)
        pred = self.linear_2(x)
        return pred
# ************* modify this section for later use *************

In [5]:
def get_activation(model, layer_name):    
    def hook(module, input, output):
        model.layer_activations[layer_name] = output
    return hook

In [6]:
def sparsity_calculator(final_spareness):
    sparseness_list_avg = list()
    sparseness_list_std = list()
    for single_epoch_spareness in final_spareness:

        hidden_layer_activation_list = single_epoch_spareness
        hidden_layer_activation_list = torch.stack(hidden_layer_activation_list)
        layer_activations_list = torch.reshape(hidden_layer_activation_list, (10000, 256))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)

        mean_sparseness_per_epoch = torch.mean(population_sparseness)
        std_sparseness_per_epoch  = torch.std(population_sparseness)

        sparseness_list_avg.append(mean_sparseness_per_epoch)
        sparseness_list_std.append(std_sparseness_per_epoch)

    return sparseness_list_avg, sparseness_list_std

In [7]:
def selectivity(hidden_layer_each_neuron):
    __selectivity__ = list()
    # I will now try to find the average of each class for each neuron.
    # check out the next cell 
    avg_activations = [dict() for x in range(256)]
    for i, neuron in enumerate(hidden_layer_each_neuron):
        for k, v in neuron.items():
            # v is the list of activations for hidden layer's neuron k 
            avg_activations[i][k] = sum(v) / float(len(v))

    # generate 256 lists to get only values in avg_activations
    only_activation_vals = [list() for x in range(256)]

    # get only values from avg_activations
    for i, avg_activation in enumerate(avg_activations):
        for value in avg_activation.values():
            only_activation_vals[i].append(value)

    for activation_val in only_activation_vals:
        # find u_max 
        u_max = np.max(activation_val)

        # find u_minus_max 
        u_minus_max = (np.sum(activation_val) - u_max) / 9

        # find selectivity 
        selectivity = (u_max - u_minus_max) / (u_max + u_minus_max)

        # append selectivity value to selectivity
        __selectivity__.append(selectivity)

    avg_selectivity = np.average(__selectivity__)
    std_selectivity = np.std(__selectivity__)
                                 
    return avg_selectivity, std_selectivity

In [8]:
def model_factory(optimizer_name, seed_num):
    '''
    optimizer_name : choose one of Adagrad, Adadelta, SGD, and Adam 

    '''
    my_model = Model()
    print("my_model:", my_model)
    my_model.to(device)

    # ************* modify this section for later use *************
    my_model.sigmoid12.register_forward_hook(get_activation(my_model, 's12'))
    # ************* modify this section for later use *************

    if optimizer_name == 'Adadelta':
        my_optimizer = torch.optim.Adadelta(my_model.parameters(), lr=1.0)

    elif optimizer_name == 'Adagrad':
        my_optimizer = torch.optim.Adagrad(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'SGD':
        my_optimizer = torch.optim.SGD(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'Adam':
        my_optimizer = torch.optim.Adam(my_model.parameters(), lr=0.001)

    else:
        print("ERROR")
    
    print("my_optimizer:", my_optimizer)
    test_acc, sparsity_avg, sparsity_std, selectivity_list_avg, selectivity_list_std = sparsity_selectivity_trainer(optimizer=my_optimizer, model=my_model)

    # ************* modify this section for later use *************
    file_saver = open(f"seed_{seed_num}_batchsize_500_50_{optimizer_name}.txt", "w")
    # ************* modify this section for later use *************
    file_saver.write(str(test_acc)+'\n'+str(sparsity_avg)+'\n'+str(sparsity_std)+'\n'+str(selectivity_list_avg)+'\n'+str(selectivity_list_std)+'\n\n')
    file_saver.close()

    if seed_num == 1:
    # ************* modify this section for later use *************
        if optimizer_name == 'Adadelta':
            !cp seed_1_batchsize_500_50_Adadelta.txt /content/drive/MyDrive
        
        elif optimizer_name == 'Adagrad':
            !cp seed_1_batchsize_500_50_Adagrad.txt /content/drive/MyDrive

        elif optimizer_name == 'SGD':
            !cp seed_1_batchsize_500_50_SGD.txt /content/drive/MyDrive

        elif optimizer_name == 'Adam':
            !cp seed_1_batchsize_500_50_Adam.txt /content/drive/MyDrive
        # ************* modify this section for later use *************
    
    elif seed_num == 100:
        # ************* modify this section for later use *************
        if optimizer_name == 'Adadelta':
            !cp seed_100_batchsize_500_50_Adadelta.txt /content/drive/MyDrive
        
        elif optimizer_name == 'Adagrad':
            !cp seed_100_batchsize_500_50_Adagrad.txt /content/drive/MyDrive

        elif optimizer_name == 'SGD':
            !cp seed_100_batchsize_500_50_SGD.txt /content/drive/MyDrive

        elif optimizer_name == 'Adam':
            !cp seed_100_batchsize_500_50_Adam.txt /content/drive/MyDrive
        # ************* modify this section for later use *************

    elif seed_num == 1234:
        # ************* modify this section for later use *************
        if optimizer_name == 'Adadelta':
            !cp seed_1234_batchsize_500_50_Adadelta.txt /content/drive/MyDrive
        
        elif optimizer_name == 'Adagrad':
            !cp seed_1234_batchsize_500_50_Adagrad.txt /content/drive/MyDrive

        elif optimizer_name == 'SGD':
            !cp seed_1234_batchsize_500_50_SGD.txt /content/drive/MyDrive

        elif optimizer_name == 'Adam':
            !cp seed_1234_batchsize_500_50_Adam.txt /content/drive/MyDrive
        # ************* modify this section for later use *************

    else:
        print("ERROR")

In [9]:
# ************* modify this section for later use *************
def avg_std_calculator(_hidden_layer_each_neuron_12):

    avg_selectivity12, std_selectivity12 = selectivity(_hidden_layer_each_neuron_12)

    final_selectivity_avg = (avg_selectivity12) / 1
    final_selecvitity_std = (std_selectivity12) / 1

    return final_selectivity_avg, final_selecvitity_std
# ************* modify this section for later use *************

In [10]:
no_epochs = 30
def sparsity_selectivity_trainer(optimizer, model):

    criterion = torch.nn.CrossEntropyLoss()
    train_loss = list()
    test_loss  = list()
    test_acc   = list()

    best_test_loss = 1

    selectivity_avg_list = list()
    selectivity_std_list = list()

    # ************* modify this section for later use *************
    final_spareness_12 = list()
    # ************* modify this section for later use *************

    for epoch in range(no_epochs):

        # ************* modify this section for later use *************
        hidden_layer_activation_list_12 = list()
        # ************* modify this section for later use *************

        # ************* modify this section for later use *************
        hidden_layer_each_neuron_12 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        hidden_layer_each_neuron_12 = np.array(hidden_layer_each_neuron_12)
        # ************* modify this section for later use *************


        total_train_loss = 0
        total_test_loss = 0

        # training
        # set up training mode 
        model.train()

        for itr, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            pred = model(images)

            loss = criterion(pred, labels)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (images, labels) in enumerate(test_dataloader):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)

            loss = criterion(pred, labels)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if labels[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            # ***************** sparsity calculation ***************** #
            hidden_layer_activation_list_12.append(model.layer_activations['s12'])
            # ***************** sparsity calculation ***************** #

            # ************* modify this section for later use *************
            for activation, label in zip(model.layer_activations['s12'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(256):    
                    hidden_layer_each_neuron_12[i][label].append(activation[i])
    
        selectivity_avg, selecvitity_std = avg_std_calculator(hidden_layer_each_neuron_12)
        # ************* modify this section for later use *************

        # this conains activations for all epochs 
        final_spareness_12.append(hidden_layer_activation_list_12)
        # ***************** sparsity calculation ***************** #
        
        selectivity_avg_list.append(selectivity_avg)
        selectivity_std_list.append(selecvitity_std)

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

    # ***************** sparsity calculation ***************** #
    sparsity_list12_avg, sparsity_list12_std = sparsity_calculator(final_spareness_12)

    average_sparsity = list()
    std_sparsity = list()

    for i in range(no_epochs):
        average_sparsity.append( (sparsity_list12_avg[i].item()) / 1 )
        std_sparsity.append( (sparsity_list12_std[i].item()) / 1 )
    # ***************** sparsity calculation ***************** #

    return test_acc, average_sparsity, std_sparsity, selectivity_avg_list, selectivity_std_list

In [11]:
# Seed 1

torch.manual_seed(1)
np.random.seed(1)

model_factory('Adadelta',1)
model_factory('Adagrad',1)
model_factory('SGD',1)
model_factory('Adam',1)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 1.09543106, Test Loss: 0.49675446, Test Accuracy: 0.87850000

Epoch: 2/30, Train Loss: 0.42427691, Test Loss: 0.35070712, Test Accuracy: 0.90100000

Epoch: 3/30, Train Loss: 0.34366980, Test Loss: 0.31398918, Test Accuracy: 0.90820000

Epoch: 4/30, Train Loss: 0.31041391, Test Loss: 0.28766843, Test Accuracy: 0.91870000

Epoch: 5/30, Train Loss: 0.28857938, Test Loss: 0.27521795, Test Accuracy: 0.91950000

Epoch: 6/30, Train Loss: 0.27015530, Test Loss: 0.25273274, Test Accuracy: 0.92770000

Epoch: 7/30, Train Loss: 0.25451269, Test Loss: 0.24370946, Test Accuracy: 0.93070000

Epoch: 8/30, Train Loss: 0.23952725, Test Loss: 0.22629044, Test Accuracy: 0.93430000

Epoch: 9/30, Train Lo

In [12]:
# Seed 100

torch.manual_seed(100)
np.random.seed(100)

model_factory('Adadelta',100)
model_factory('Adagrad',100)
model_factory('SGD',100)
model_factory('Adam',100)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 1.08656033, Test Loss: 0.49473005, Test Accuracy: 0.88000000

Epoch: 2/30, Train Loss: 0.42238409, Test Loss: 0.34750127, Test Accuracy: 0.90110000

Epoch: 3/30, Train Loss: 0.34199817, Test Loss: 0.30791468, Test Accuracy: 0.91290000

Epoch: 4/30, Train Loss: 0.30962214, Test Loss: 0.28979171, Test Accuracy: 0.91900000

Epoch: 5/30, Train Loss: 0.28951365, Test Loss: 0.26724598, Test Accuracy: 0.92080000

Epoch: 6/30, Train Loss: 0.27240706, Test Loss: 0.25772192, Test Accuracy: 0.92580000

Epoch: 7/30, Train Loss: 0.25688292, Test Loss: 0.24386058, Test Accuracy: 0.93120000

Epoch: 8/30, Train Loss: 0.24239181, Test Loss: 0.23111051, Test Accuracy: 0.93310000

Epoch: 9/30, Train Lo

In [13]:
# Seed 1234

torch.manual_seed(1234)
np.random.seed(1234)

model_factory('Adadelta',1234)
model_factory('Adagrad',1234)
model_factory('SGD',1234)
model_factory('Adam',1234)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 1.08014344, Test Loss: 0.49910621, Test Accuracy: 0.87560000

Epoch: 2/30, Train Loss: 0.42347712, Test Loss: 0.35971607, Test Accuracy: 0.89730000

Epoch: 3/30, Train Loss: 0.34335974, Test Loss: 0.31458733, Test Accuracy: 0.90880000

Epoch: 4/30, Train Loss: 0.31070624, Test Loss: 0.28962053, Test Accuracy: 0.91600000

Epoch: 5/30, Train Loss: 0.28905155, Test Loss: 0.26851619, Test Accuracy: 0.92210000

Epoch: 6/30, Train Loss: 0.27119273, Test Loss: 0.25438638, Test Accuracy: 0.92700000

Epoch: 7/30, Train Loss: 0.25639339, Test Loss: 0.24526014, Test Accuracy: 0.92710000

Epoch: 8/30, Train Loss: 0.24147533, Test Loss: 0.22815686, Test Accuracy: 0.93360000

Epoch: 9/30, Train Lo